In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [4]:
spark = SparkSession.builder.appName("Refine tse 2022").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/08 07:52:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
df = spark.read.csv('/home/guilherme.cunha/Área de Trabalho/WORKSPACE/Compass/Testes/2022.csv', header=True, inferSchema=True,sep=';')

In [8]:
df.show()

+-----+--------+----------+--------------------+-----------------+-------------------+
|sg_uf|nr_turno|  ds_cargo|        nm_candidato|qt_votos_nominais|           dt_carga|
+-----+--------+----------+--------------------+-----------------+-------------------+
|   AC|       2|Presidente|JAIR MESSIAS BOLS...|           287750|2022-11-27 18:56:18|
|   AC|       2|Presidente|LUIZ INACIO LULA ...|           121566|2022-11-27 18:56:18|
|   AL|       2|Presidente|JAIR MESSIAS BOLS...|           687827|2022-11-27 18:56:18|
|   AL|       2|Presidente|LUIZ INACIO LULA ...|           976831|2022-11-27 18:56:18|
|   AM|       2|Presidente|JAIR MESSIAS BOLS...|           961741|2022-11-27 18:56:18|
|   AM|       2|Presidente|LUIZ INACIO LULA ...|          1004991|2022-11-27 18:56:18|
|   AP|       2|Presidente|JAIR MESSIAS BOLS...|           200547|2022-11-27 18:56:18|
|   AP|       2|Presidente|LUIZ INACIO LULA ...|           189918|2022-11-27 18:56:18|
|   BA|       2|Presidente|JAIR MESSIAS BOL

In [15]:
total = df.groupBy('sg_uf').sum('qt_votos_nominais').withColumnRenamed("sum(qt_votos_nominais)", "total_votos").sort('sg_uf')
total = total.withColumnRenamed("sg_uf", "uf_drop")

In [16]:
df = df.join(total, df.sg_uf == total.uf_drop,'left').drop('uf_drop','ds_cargo','dt_carga','nr_turno')

In [17]:
df = df.withColumn('porcentagem', (col('qt_votos_nominais') * 100)/col('total_votos')).withColumn('year', lit(2022))

In [18]:
df.show()

+-----+--------------------+-----------------+-----------+------------------+----+
|sg_uf|        nm_candidato|qt_votos_nominais|total_votos|       porcentagem|year|
+-----+--------------------+-----------------+-----------+------------------+----+
|   AC|JAIR MESSIAS BOLS...|           287750|     409316| 70.30020815213673|2022|
|   AC|LUIZ INACIO LULA ...|           121566|     409316|29.699791847863263|2022|
|   AL|JAIR MESSIAS BOLS...|           687827|    1664658| 41.31941816276977|2022|
|   AL|LUIZ INACIO LULA ...|           976831|    1664658| 58.68058183723023|2022|
|   AM|JAIR MESSIAS BOLS...|           961741|    1966732| 48.90046025589658|2022|
|   AM|LUIZ INACIO LULA ...|          1004991|    1966732| 51.09953974410342|2022|
|   AP|JAIR MESSIAS BOLS...|           200547|     390465| 51.36106949406477|2022|
|   AP|LUIZ INACIO LULA ...|           189918|     390465| 48.63893050593523|2022|
|   BA|JAIR MESSIAS BOLS...|          2357028|    8454843| 27.87784468617572|2022|
|   

In [ ]:
df.withColumn("Winner")

In [19]:
df.write.mode("append").partitionBy("year","sg_uf",'nm_candidato') \
    .parquet('/home/guilherme.cunha/Área de Trabalho/WORKSPACE/Compass/Testes/teste')